# 4. Building a Chatbot

## Setup

In [19]:
%pip install langgraph langchain-tavily langgraph-checkpoint-sqlite

import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

assert os.environ["LANGSMITH_TRACING"] is not None
assert os.environ["LANGSMITH_API_KEY"] is not None
assert os.environ["LANGSMITH_PROJECT"] is not None
assert os.environ["OPENAI_API_KEY"] is not None
assert os.environ["TAVILY_API_KEY"] is not None


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o-mini", model_provider="openai")

## 4.1 Introduction to Agents

- An LLM agent is a system that uses a large language model to **autonomously decide which actions or tools to invoke in order to achieve a defined goal**.
- In this section we are going to build an agent that is capable of performing web searches using a tool called Tavily. We will combine this with the chatbot functionality in the previous chapter.

## 4.1.1 Introduction to Tavily

- Tavily is a search API designed specifically for AI and LLM applications.
- Instead of crawling the whole web like Google, it provides focused, structured, and high-relevance results that are easy for models to consume.

In [21]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco, CA', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1758093132, 'localtime': '2025-09-17 00:12'}, 'current': {'last_updated_epoch': 1758092400, 'last_updated': '2025-09-17 00:00', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 4.9, 'wind_kph': 7.9, 'wind_degree': 254, 'wind_dir': 'WSW', 'pressure_mb': 1015.0, 'pressure_in': 29.96, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 90, 'cloud': 25, 'feelslike_c': 17.2, 'feelslike_f': 63.0, 'windchill_c': 14.4, 'windchill_f': 58.0, 'heatindex_c': 14.5, 'heatindex_f': 58.0, 'dewpoint_c': 13.8, 'dewp

## 4.1.2 Introduction to Tools

- LLM tools are external functions or services that a large language model can call—such as search, databases, or APIs—to extend its capabilities beyond text generation.
- To enable a model to use tools, we use the `bind_tools` method:

In [22]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

model = init_chat_model("gpt-4.1", model_provider="openai")
model_with_tools = model.bind_tools(tools)

For an ordinary query, we see response text and no tool calls:

In [23]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! How can I assist you today?

Tool calls: []


However: **Note** that when we bind a tool and ask a question, we get no response but we get a tool call

In [24]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'current weather in San Francisco', 'search_depth': 'basic'}, 'id': 'call_uAxirBDuGtq8dDYHUpavRfGS', 'type': 'tool_call'}]


**Notice** The model wants us to call the Tavily Search tool. In order for the model to call the agent autonomously, we need to build an agent

## 4.2 Building an Agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using LangGraph to construct the agent. Currently, we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.

In [26]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I help you today?


In [27]:
input_message = {"role": "user", "content": "Search for the weather in SF"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_bbxdZf2JFqDL2U4WLo4iGR0I)
 Call ID: call_bbxdZf2JFqDL2U4WLo4iGR0I
  Args:
    query: current weather in San Francisco
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1758095625, 'localtime': '2025-09-17 00:53'}, 'current': {'last_updated_epoch': 1758095100, 'last_updated': '2025-09-17 00:45', 'temp_c': 16.1, 'temp_f': 61.0, 'is

### 4.2.1 Streaming Messages

If the agent executes multiple steps, this may take a while. 
**To show intermediate progress, we can stream back messages as they occur.**

In [29]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_Fd3vOICozgLNJZ5E0sJEHh0e)
 Call ID: call_Fd3vOICozgLNJZ5E0sJEHh0e
  Args:
    query: current weather in San Francisco
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1758095625, 'localtime': '2025-09-17 00:53'}, 'current': {'last_updated_epoch': 1758095100, 'last_updated': '2025-09-17 00:45', 'temp_c': 1

### 4.2.2 Streaming Tokens

In addition to streaming back messages, it is also useful to stream back tokens. We can do this by specifying stream_mode="messages".

In [30]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

NameError: name 'config' is not defined

### 4.2.3 Adding Memory

In order for us to be able to chat with the agent, we need to add memory.

In [31]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [("user", "Hi, I'm Bob!")]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================

Hello Bob! How can I help you today?


In [32]:
for step in agent_executor.stream(
    {"messages": [("user", "What is my name?")]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is my name?
================================== Ai Message ==================================

Your name is Bob! How can I assist you further?
